# Model testing and choosing
trying 5 different regressions with 4 different scalers to choose the best one to tune up.



In [ ]:
import pandas as pd
import numpy as np
#import seaborn as sns
import random
import pickle


In [ ]:

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:

import matplotlib.pyplot as plt
from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# from google.colab import files

# #define function to save models locally after they have run
# def save_model(model, string):
#   filename = string + '.sav'
#   pickle.dump(model, open(filename, 'wb'))
#   files.download(filename)
  
  
#   print(filename, ' has been saved to drive!')

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Preprocessing 

In [ ]:
#import file
data= pd.read_csv('gdrive/My Drive/Colab Notebooks/county_df.csv')
data.drop(columns = "smallbrew_per_capita", inplace = True)

In [ ]:
data.set_index(['Area','County','Year'], inplace = True)
#convert all to numeric
data = data.apply(pd.to_numeric, errors = 'coerce')
#save dataframe for later just i
save = data


In [ ]:
data.reset_index(inplace = True)

In [ ]:

from sklearn.preprocessing import QuantileTransformer, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
#from sklearn.imputer import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, mean_absolute_error
def adjusted_r2(train, r2):
  adj_r2 = 1 - (1 - r2) * ((train.shape[0] - 1) / (train.shape[0] - train.shape[1] - 1))
  return adj_r2



In [ ]:
def scaler_test(model, model_string, scalers, scaler_strings, data_X, data_y, results_list):
  
  for i in range(len(scalers)):
    
    #scale data_X
    X_scaled = scalers[i].fit_transform(data_X)
    
    #split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, data_y, test_size=0.2, random_state=42)

    #fit model on training data
    model.fit(X_train, y_train)

    #test on testing data
    y_pred = model.predict(X_test)
  
    #calculate metrics 

    r2 = r2_score(y_test, y_pred)
    adj_r2 = adjusted_r2(X_train, r2)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)

    results.append([model_string,scaler_strings[i], adj_r2, rmse, mae])
    
    print('Results for ', model_string, ' using ', scaler_strings[i])
    print('adjusted r2: %.2f' % adj_r2)
    print('RMSE: %.2f' % rmse)
    print('MAE: %.2f' % mae)
    print('...................')
    

###Elastic Net

In [141]:
results = []

enet = linear_model.ElasticNet()
data_X = data.drop(columns = ['Area', 'Breweries_Small'])
data_X = data_X.apply(pd.to_numeric, errors = 'coerce').astype(float)
data_y = data['Breweries_Small']

scalers = [minmax, ss, robust, qt]
scaler_strings = ['minmax', 'ss', 'robust', 'qt']

scaler_test(enet, 'enet', scalers, scaler_strings, data_X, data_y, results)

Results for  enet  using  minmax
adjusted r2: -0.01
RMSE: 2.84
MAE: 1.17
...................
Results for  enet  using  ss
adjusted r2: 0.58
RMSE: 1.82
MAE: 0.83
...................
Results for  enet  using  robust
adjusted r2: 0.66
RMSE: 1.65
MAE: 0.77
...................
Results for  enet  using  qt
adjusted r2: -0.01
RMSE: 2.84
MAE: 1.17
...................


### Ridge Regression: 

In [142]:
ridge = linear_model.Ridge()

#data_X = data.drop(columns = ['Area', 'Breweries_Small'])
#data_y = data['Breweries_Small']

scalers = [minmax, ss, robust, qt]
scaler_strings = ['minmax', 'ss', 'robust', 'qt']

scaler_test(ridge, 'ridge', scalers, scaler_strings, data_X, data_y, results)

Results for  ridge  using  minmax
adjusted r2: 0.76
RMSE: 1.39
MAE: 0.74
...................
Results for  ridge  using  ss
adjusted r2: 0.76
RMSE: 1.40
MAE: 0.74
...................
Results for  ridge  using  robust
adjusted r2: 0.76
RMSE: 1.40
MAE: 0.74
...................
Results for  ridge  using  qt
adjusted r2: 0.47
RMSE: 2.05
MAE: 0.91
...................


### Linear Regression: 

In [143]:
linreg = linear_model.LinearRegression()

#data_X = data.drop(columns = ['Area', 'Breweries_Small'])
#data_y = data['Breweries_Small']

scalers = [minmax, ss, robust, qt]
scaler_strings = ['minmax', 'ss', 'robust', 'qt']

scaler_test(linreg, 'linreg', scalers, scaler_strings, data_X, data_y, results)

Results for  linreg  using  minmax
adjusted r2: 0.76
RMSE: 1.40
MAE: 0.75
...................
Results for  linreg  using  ss
adjusted r2: 0.76
RMSE: 1.40
MAE: 0.74
...................
Results for  linreg  using  robust
adjusted r2: 0.75
RMSE: 1.40
MAE: 0.74
...................
Results for  linreg  using  qt
adjusted r2: 0.47
RMSE: 2.05
MAE: 0.91
...................


### SVM

In [145]:
from sklearn.svm import SVR
svr = SVR()

scalers = [minmax, ss, robust, qt]
scaler_strings = ['minmax', 'ss', 'robust', 'qt']

scaler_test(svr, 'svr', scalers, scaler_strings, data_X, data_y, results)

Results for  svr  using  minmax
adjusted r2: 0.42
RMSE: 2.16
MAE: 0.75
...................
Results for  svr  using  ss
adjusted r2: 0.23
RMSE: 2.48
MAE: 0.73
...................
Results for  svr  using  robust
adjusted r2: 0.22
RMSE: 2.49
MAE: 0.72
...................
Results for  svr  using  qt
adjusted r2: 0.33
RMSE: 2.31
MAE: 0.74
...................


### Random Forests

In [ ]:
data_y = data.Breweries_Small
data_X = data.drop(['Breweries_Small', 'Area'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=42)

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)
y_pred = forest_reg.predict(X_test)

In [148]:
#calculate metrics 

r2 = r2_score(y_test, y_pred)
adj_r2 = adjusted_r2(X_train, r2)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

results.append(['forest','none', adj_r2, rmse, mae])

print('adjusted r2: %.2f' % adj_r2)
print('RMSE: %.2f' % rmse)
print('MAE: %.2f' % mae)



adjusted r2: 0.75
RMSE: 1.41
MAE: 0.61


In [ ]:
columns = ['Model', 'scaler', 'adjusted_r2', 'RMSE', 'MSE']
results_df = pd.DataFrame(results, columns = columns)

In [153]:
top = results_df.loc[results_df.groupby("Model")["MSE"].idxmin()]
top.set_index('Model', inplace = True)
top.sort_values('MSE')

,scaler,adjusted_r2,RMSE,MSE
Model,,,,
forest,none,0.749962,1.412813,0.607421
svr,robust,0.220482,2.494564,0.719835
ridge,minmax,0.758974,1.387118,0.740121
linreg,robust,0.752744,1.404931,0.743074
enet,robust,0.658944,1.650039,0.766556


#### random search cv

In [ ]:
data_Y = data.Breweries_Small
data_X = data.drop('Breweries_Small', axis = 1)
ss_scaler = StandardScaler()
data_X = ss_scaler.fit_transform(data_X)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=42)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
forest_reg = RandomForestRegressor()

In [ ]:
random_search = RandomizedSearchCV(forest_reg, random_grid,n_iter = 20, cv = 5, verbose=4, random_state=42, n_jobs = -1)

In [ ]:
random_search.fit(X_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.5min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': [20, 32, 45, 58, 71, 84, 97, 110, 122, 135, 148, 161, 174, 187, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [2, 4, 6], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=4)

In [ ]:
random_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=30,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=32, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
save_model(rand_forest1, 'rand_forest1')

rand_forest1.sav  has been saved to drive!


In [ ]:
rand_forest1 = random_search.best_estimator_

In [ ]:
y_pred = rand_forest1.predict(X_test)

In [ ]:
print('r2 score: %.2f' % r2_score(y_test, y_pred))

mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

print('mse:',mse,'rmse:', rmse)

print('mean_error:',mean_absolute_error(y_test, y_pred))

print('median_error:' , median_absolute_error(y_test, y_pred))

r2 score: 0.85
mse: 0.8246711570344025 rmse: 0.9081140660921416
mean_error: 0.5701013832010858
median_error: 0.34357638888888886


In [ ]:
pca = decomposition.PCA(n_components = 7)
data_X = pca.fit_transform(data_X)
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=42)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
rand_forest2 = random_search.best_estimator_
y_pred = rand_forest2.predict(X_test)
print('r2 score: %.2f' % r2_score(y_test, y_pred))

mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

print('mse:',mse,'rmse:', rmse)

print('mean_error:',mean_absolute_error(y_test, y_pred))

print('median_error:' , median_absolute_error(y_test, y_pred))

r2 score: 0.67
mse: 1.7760819451034653 rmse: 1.332697244352019
mean_error: 0.6692210707558803
median_error: 0.3877106314736656
